In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

In [ ]:
telecom_df = pd.read_csv('telecom_churn_data.csv')
print(telecom_df.head(5))
print(telecom_df.shape) #(99999, 226)
print(telecom_df.info()) 

## Preprocessing

#### Date column formatting

In [ ]:
#print(telecom_df.select_dtypes(include=['object']).head())
#These are all of type data, of format mm/dd/yyyy, conver to dates
#for all date columns, keep only the day of the month, since month is present in the
#column name, year is common for all.
date_columns = list(telecom_df.select_dtypes(include=['object']).columns)
for column in date_columns:
    telecom_df[column] = pd.to_datetime(telecom_df[column], format='%m/%d/%Y')
    telecom_df[column] = telecom_df[column].dt.day

#### Identify Redundant columns

In [ ]:
#Find columns with only single non-null value
redundant_cols = [col for col in telecom_df.columns if telecom_df[col].nunique()==1]
telecom_df = telecom_df.drop(redundant_cols, axis=1)
print('Following columns were removed: \n',redundant_cols)

#### Identifying the top 30 percentile in terms of total recharge done in month 6 and 7 (good phase)

In [ ]:
#Those who have recharged with an amount more than or equal to X, 
#where X is the 70th percentile of the average recharge amount in the first two months (the good phase).
'''
print(telecom_df['total_rech_amt_6'].isnull().sum()) 0;
print(telecom_df['total_rech_amt_7'].isnull().sum()) 0;
print(telecom_df['total_rech_num_6'].isnull().sum()) 0;
print(telecom_df['total_rech_num_7'].isnull().sum()) 0;
'''
#since, all values for the columns used below are non-nulls, we need not do any imputation here
if 'av_rech_67' not in list(telecom_df.columns):
    telecom_df['av_rech_67'] =  (telecom_df['total_rech_amt_6']+telecom_df['total_rech_amt_7'])/(telecom_df['total_rech_num_6']+telecom_df['total_rech_num_7'])
    avg_70_quantile = telecom_df.av_rech_67.quantile(q=0.7)
    telecom_df = telecom_df[telecom_df.av_rech_67>= avg_70_quantile]
print ('Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: '+ str(len(telecom_df)))

#### Null value analysis


In [ ]:
telecom_df.isnull().sum().describe()
# telecom_df.isnull().sum()>0).sum()
# 166 columns with missing values
# A lot of values are missing in many colums. Cannot remove such rows. Will have to find a way to impute

##### Nullvalue imputation for internet-data based columns

In [ ]:
for month in [6,7,8,9]:
    max_rech_data = 'max_rech_data_'+str(month)
    av_rech_amt_data='av_rech_amt_data_'+str(month)
    total_rech_data = 'total_rech_data_'+str(month)
    date_of_last_rech_data = 'date_of_last_rech_data_'+str(month)
    count_rech_3g = 'count_rech_3g_'+str(month)
    count_rech_2g = 'count_rech_2g_'+str(month)
    arpu_3g = 'arpu_3g_'+str(month)
    arpu_2g = 'arpu_2g_'+str(month)
    fb_user = 'fb_user_'+str(month)
    night_pck_user = 'night_pck_user_' +str(month)

    #Verify, that if toatl rech_data is null, then max, av, date, count_3g, count_2g are null too 
    #nonnullrows = np.sum(np.sum(telecom_df[telecom_df[total_rech_data].isnull()]
    #if last date for data recharge is null, then verify if other elements corresponding to null are null/0 too. If yes,
    # we can assign all of them as 0.
    
    max_rech_nonzerorows = len(telecom_df[(telecom_df[date_of_last_rech_data].isnull()) & ((~telecom_df[max_rech_data].isnull()) | (telecom_df[max_rech_data]>0))])
    if max_rech_nonzerorows == 0:
        telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),max_rech_data] = 0

    #Similarily for rest columns too
    assumption=True
    for data_colm in [total_rech_data, av_rech_amt_data,count_rech_3g,count_rech_2g,arpu_3g,arpu_2g,fb_user,night_pck_user]:
        rech_colm_nonzerorows = len(telecom_df[(telecom_df[date_of_last_rech_data].isnull()) & ((~telecom_df[data_colm].isnull()) | (telecom_df[data_colm]>0))])
        if rech_colm_nonzerorows == 0:
            telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),data_colm] = 0
        else:
            assumption = False
            print('Wrong assumption for', data_colm)
    if assumption == True:
#       If assumption was correct, then provide invalid date, or -ve
        telecom_df.loc[telecom_df[date_of_last_rech_data].isnull(),date_of_last_rech_data] = -1

##### Nullvalue imputation for basic recharge based columns

In [ ]:
def makeZeroIfTotalIsZero(colm, total_colm, typeofcol):
    total_ = np.sum(telecom_df[telecom_df[colm].isnull()][total_colm])
    if total_ == 0:
        #('Total for ', typeofcol, ' is 0. Hence make all null values 0')
        telecom_df.loc[telecom_df[colm].isnull(),colm] = 0

for month in [6,7,8,9]:
    total_ic_mou = 'total_ic_mou_'+str(month)
    roam_ic_mou = 'roam_ic_mou_'+str(month)
    loc_ic_t2t_mou = 'loc_ic_t2t_mou_'+str(month)
    loc_ic_t2m_mou = 'loc_ic_t2m_mou_'+str(month)
    loc_ic_t2f_mou = 'loc_ic_t2f_mou_'+str(month)
    loc_ic_mou  = 'loc_ic_mou_'+str(month)
    std_ic_t2t_mou = 'std_ic_t2t_mou_'+str(month)
    std_ic_t2m_mou = 'std_ic_t2m_mou_'+str(month)
    std_ic_t2f_mou = 'std_ic_t2f_mou_'+str(month)
    std_ic_mou  =  'std_ic_mou_'+str(month)
    spl_ic_mou = 'spl_ic_mou_'+str(month)
    isd_ic_mou = 'isd_ic_mou_'+str(month)
    ic_others  = 'ic_others_'+str(month)

    total_og_mou = 'total_og_mou_' + str(month)
    roam_og_mou = 'roam_og_mou_' + str(month)
    loc_og_t2t_mou = 'loc_og_t2t_mou_' + str(month)
    loc_og_t2m_mou = 'loc_og_t2m_mou_' + str(month)
    loc_og_t2f_mou = 'loc_og_t2f_mou_' + str(month)
    loc_og_t2c_mou = 'loc_og_t2c_mou_' + str(month)
    loc_og_mou = 'loc_og_mou_' + str(month)
    std_og_t2t_mou = 'std_og_t2t_mou_' + str(month)
    std_og_t2m_mou = 'std_og_t2m_mou_' + str(month)
    std_og_t2f_mou = 'std_og_t2f_mou_' + str(month)
    std_og_mou = 'std_og_mou_' + str(month)
    isd_og_mou = 'isd_og_mou_' + str(month)
    spl_og_mou = 'spl_og_mou_' + str(month)
    og_others = 'og_others_' + str(month)
      
    onnet_mou = 'onnet_mou_'+str(month)
    offnet_mou = 'offnet_mou_'+str(month)

    date_of_last_rech = 'date_of_last_rech_' + str(month)
    total_rech_amt = 'total_rech_amt_'+ str(month)
    #Incoming columns
    for colm in [roam_ic_mou, loc_ic_t2t_mou, loc_ic_t2m_mou, 
                 loc_ic_t2f_mou, loc_ic_mou, std_ic_t2t_mou, 
                 std_ic_t2m_mou, std_ic_t2f_mou, std_ic_mou,
                 spl_ic_mou, isd_ic_mou, ic_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'incoming')
    
    #Outgoing columns

    for colm in [roam_og_mou,loc_og_t2t_mou,loc_og_t2m_mou,
                 loc_og_t2f_mou,loc_og_t2c_mou,loc_og_mou,
                 std_og_t2t_mou,std_og_t2m_mou,std_og_t2f_mou,
                 std_og_mou,isd_og_mou,spl_og_mou,og_others]:
        makeZeroIfTotalIsZero(colm, total_ic_mou, 'outgoing')
    
    #Onnet columns
    #If outgoing + incoming mou for the nuxll entries is 0, then onnet, offnet mou are also 0
    outgoing_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_og_mou])
    incoming_mou_when_onnet_null = np.sum(telecom_df[telecom_df[onnet_mou].isnull()][total_ic_mou])
    onnet = outgoing_mou_when_onnet_null+incoming_mou_when_onnet_null
    #'Total onnet(calculated) for month: ', str(month), ' is: ', onnet, ' Actual null'..... #0
    telecom_df.loc[telecom_df[onnet_mou].isnull(),onnet_mou] = 0
    
    outgoing_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_og_mou])
    incoming_mou_when_offnet_null = np.sum(telecom_df[telecom_df[offnet_mou].isnull()][total_ic_mou])
    offnet = outgoing_mou_when_offnet_null+incoming_mou_when_offnet_null
    #'Total offnet(calculated) for month: ', str(month), ' is: ', offnet, ' Actual null'.... #0
    telecom_df.loc[telecom_df[offnet_mou].isnull(),offnet_mou] = 0
    
    
    total_ = np.sum(telecom_df[telecom_df[date_of_last_rech].isnull()][total_rech_amt])
    if total_ == 0:
        telecom_df.loc[telecom_df[date_of_last_rech].isnull(),date_of_last_rech] = -1

In [ ]:
#Verify there are no null values remaining
print (0 == (telecom_df.isnull().sum()>0).sum()) # True

## Feature Engineering

### 1. Target variable (Churn)

In [ ]:
#Identify churners now
telecom_df['churn'] = np.zeros(len(telecom_df))
telecom_df.loc[((telecom_df['total_ic_mou_9'] == 0) & 
                           (telecom_df['total_og_mou_9'] == 0) &
                           (telecom_df['vol_2g_mb_9'] == 0) &
                           (telecom_df['vol_3g_mb_9'] == 0) 
               ),'churn'] = 1
#Remove everything with _9
colms_with_9 = [colm for colm in telecom_df.columns if colm.endswith('_9')]
telecom_df = telecom_df.drop(colms_with_9, axis=1)

### 2. Recency variables

In [ ]:
telecom_df['recency_goodphase'] = np.ones(len(telecom_df))*61
telecom_df['recency_goodphase_data'] = np.ones(len(telecom_df))*61

#Regular recharge
telecom_df.loc[telecom_df.date_of_last_rech_7>0,'recency_goodphase']= 31 - telecom_df[telecom_df.date_of_last_rech_7 > 0].date_of_last_rech_7
telecom_df.loc[(telecom_df.date_of_last_rech_7<0)&(telecom_df.date_of_last_rech_6>0),'recency_goodphase']= 31 + (30 - telecom_df[(telecom_df.date_of_last_rech_7<0)&(telecom_df.date_of_last_rech_6 > 0)].date_of_last_rech_6)
                                                                            
                 
#Data recharge
telecom_df.loc[telecom_df.date_of_last_rech_data_7>0,'recency_goodphase_data']= 31 - telecom_df[telecom_df.date_of_last_rech_data_7 > 0].date_of_last_rech_data_7
telecom_df.loc[(telecom_df.date_of_last_rech_data_7<0)&(telecom_df.date_of_last_rech_data_6>0),'recency_goodphase_data']= 31 + (30 - telecom_df[(telecom_df.date_of_last_rech_data_7<0)&(telecom_df.date_of_last_rech_data_6 > 0)].date_of_last_rech_data_6) 

### 3. Data volume consumed per rupee of recharge

In [ ]:
#Data Volume of data used per rupee. (2gvol+3gvol)/av_rech_amt_data
for month in [6,7,8]:
    data_volm_used_per_rupee = 'data_volm_used_per_rupee_'+str(month)
    vol_3g_mb = 'vol_3g_mb_'+str(month)
    vol_2g_mb = 'vol_2g_mb_'+str(month)
    av_recharge = 'av_rech_amt_data_'+str(month)
    telecom_df.loc[telecom_df[av_recharge]==0, data_volm_used_per_rupee] =0
    telecom_df.loc[telecom_df[av_recharge]>0,data_volm_used_per_rupee] = (telecom_df[vol_2g_mb] + telecom_df[vol_3g_mb])/telecom_df[av_recharge]

### 4. Data heavy/ call heavy

In [ ]:
for month in [6,7,8]: 
    colname = 'dataHeavy_'+str(month)
    total_amt_data = telecom_df['av_rech_amt_data_'+str(month)]
    total_amt_basic = telecom_df['total_rech_amt_'+str(month)]
    telecom_df[colname] = total_amt_data/total_amt_basic
    telecom_df.loc[telecom_df[colname]>1, colname] = 1
    telecom_df.loc[telecom_df[colname]<1, colname] = 0

### 5. Good-phase aggregate variables

In [ ]:
#The variables to combine in goodphase vs month8
cols_monthwise_67 =list(set(['_'.join(colm.split('_')[:-1]) for colm in telecom_df.columns if (colm.endswith('_6') or colm.endswith('_7'))]))

In [ ]:
#aggregation techniques to combine columns from month 6 and 7 to goodphase vaiables
suffix = '_goodphase'
def agg_weighted_addtion(col):
    w_7 = 0.7 
    w_6 = 0.3
    telecom_df[col+suffix] = telecom_df[col+'_6']*w_6 + telecom_df[col+'_7']*w_7

def max_assign(col):
    telecom_df[col+suffix] = np.max(telecom_df[[col+'_6',col+'_7']], axis=1)    

def direct_assign(col):
    telecom_df[col+suffix] = telecom_df[col+'_7'] #more weight for 7th month
    #telecom_df[col+suffix] = telecom_df[col+suffix].apply(lambda num: 1 if num>1 else 0)

max_cols = ['max_rech_data','max_rech_amt']
continuity_cols = ['fb_user','night_pck_user','sachet_2g','sachet_3g','monthly_2g','monthly_3g','dataHeavy','last_day_rch_amt']
numeric_cols = [ 'loc_og_t2t_mou','loc_ic_t2m_mou','loc_ic_mou','total_rech_num', \
             'onnet_mou','count_rech_3g','vol_3g_mb','roam_ic_mou','loc_og_t2m_mou', \
             'total_rech_amt','loc_og_t2f_mou','std_og_t2m_mou','loc_og_t2c_mou','std_ic_t2f_mou', \
             'loc_ic_t2t_mou', 'total_rech_data','arpu_2g','count_rech_2g','roam_og_mou', \
             'og_others','loc_og_mou','std_og_mou','ic_others','total_og_mou','std_og_t2t_mou', \
             'arpu_3g','isd_og_mou','vol_2g_mb','isd_ic_mou','std_ic_mou','std_ic_t2m_mou', \
             'total_ic_mou','spl_ic_mou','spl_og_mou','std_ic_t2t_mou','loc_ic_t2f_mou', \
             'offnet_mou', 'std_og_t2f_mou','arpu','av_rech_amt_data','data_volm_used_per_rupee']


In [ ]:
for col in numeric_cols:
    agg_weighted_addtion(col)
    
for col in continuity_cols:
    direct_assign(col)

for col in max_cols:
    max_assign(col)

for col in cols_monthwise_67:
    telecom_df = telecom_df.drop(col+'_6', axis=1)
    telecom_df = telecom_df.drop(col+'_7', axis=1)